# import  efficientnet  to wok offline

In [ ]:
import sys
sys.path.append('../input/efficientnet-keras-dataset/efficientnet_kaggle')
! pip install -e ../input/efficientnet-keras-dataset/efficientnet_kaggle

In [ ]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers



import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
import time
import json
import copy
import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics
print("Tensorflow version " + tf.__version__)

In [ ]:
import efficientnet.keras as efn 


# tpu detaction and config

ref  https://www.tensorflow.org/guide/tpu   https://www.kaggle.com/docs/tpu

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

using gcs_path

In [ ]:

AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH =  KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512 , 512]
CLASSES = ['0', '1', '2', '3', '4']
EPOCHS = 25

# working with tfrecoreds

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/ld_train*.tfrec'),
    test_size=0.25, random_state=5
)

TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')

# doing random data augmention on tpu

In [ ]:
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
    
    dc = random.choice(["flip","rotat" , "contrast" , "brightnes"])
    if dc =='flip':
        image = tf.image.random_flip_left_right(image)
    elif dc =='rotat':
        image = tf.image.random_flip_up_down(image)   
    elif dc =='contrast':
        image = tf.image.random_contrast(image, 0.2, 0.5)
    elif dc =='brightnes':
        image = tf.image.random_brightness(image , 0.2)
    
    return image, label

# datasets

In [ ]:
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE) 
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered) 
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)

print('Dataset: {} training images, {} validation images, {} (unlabeled) test images'.format(
    NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

# learning rate schudler

In [ ]:
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4, 
    decay_steps=10000, 
    decay_rate=0.9)

In [ ]:
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers 
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks

# model creation function

In [ ]:


def create_model(met,trin=0.35):

    base =  efn.EfficientNetB7(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=None,
       
    )
    trin = round(len(base.layers)*trin)
    for layer in base.layers[:-trin]:
        layer.trainable = False
    model = models.Sequential()
    '''auglayer = tf.keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                       layers.experimental.preprocessing.RandomRotation(0.2),
                        layers.experimental.preprocessing.RandomContrast(1),
                        layers.experimental.preprocessing.RandomZoom(0.2)])
    model.add(keras.Input(shape=(512,512,3,)))
    model.add(auglayer) 
    '''
    model.add(base)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.4))


    model.add(layers.Dense(512, activation='relu'))

    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(128, activation='relu'))

    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(5, activation="softmax"))

    #adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
        loss='sparse_categorical_crossentropy',  
        metrics=met)
    return model



# defin the model within tpu stratgy and set parameters

In [ ]:
with strategy.scope():
    METRICS = [ 
          keras.metrics.SparseCategoricalAccuracy(name='accuracy')
    ]
    model = create_model(met=METRICS,trin=0.5)
model.summary()

In [ ]:
# load data
train_dataset = get_training_dataset()
valid_dataset = get_validation_dataset()

In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE
#checkpoits for saving best wights based on val_loss 
mcp_save = callbacks.ModelCheckpoint('mamon-checkpoint-tpu-efnb7-5hlf.hdf5', save_best_only=True, monitor='val_loss', mode='min')

#start training while history will contain metrices
history = model.fit(train_dataset, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS,
                    validation_data=valid_dataset,
                    validation_steps=VALID_STEPS,callbacks=[mcp_save])

In [ ]:
# print out variables available to us  
print(history.history.keys())

In [ ]:
# create learning curves to evaluate model performance
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
print("Evaluate on test data")
results = model.evaluate(valid_dataset)
print("test loss, test acc:", results)

In [ ]:
model = keras.models.load_model('./mamon-checkpoint-tpu-efnb7-5hlf.hdf5')
print("Evaluate on test data")
results = model.evaluate(valid_dataset)
print("test loss, test acc:", results)

In [ ]:
model.save('efnetb7-acc88.hdf5')
